In [15]:
import numpy as np

In [16]:
class model:
    def __init__(self,layers):
        weightShape = [(a,b) for a,b in zip(layers[1:],layers[:-1])]
        self.weights =[np.random.standard_normal(shape)/shape[1]**.5 for shape in weightShape]
        self.biases = [np.zeros((c,1)) for c in layers[1:]]
    
    def predict(self,a):
        for w,b in zip(self.weights,self.biases):
            a = self.activation(np.matmul(w,a) + b)
        return a

    @staticmethod
    def activation(x):
        return 1/(1+np.exp(-x))
    
    def train(train_images,train_labels):
        